In [1]:
import os
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import string
import warnings
warnings.filterwarnings('ignore')
import string
import collections
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
os.chdir(r"C:\Users\PC\Desktop\Machine Learning\Datasets")

In [5]:
full_df = pd.read_csv("sample.csv")

In [6]:
df = full_df[["text"]]
df["text"] = df["text"].astype(str)
full_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


# Lower Casing

In [7]:
df["text_lower"] = df["text"].str.lower()
df.head()

,text,text_lower
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...


# Removal of Punctuations

In [8]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [9]:
df["text_wo_punct"] = df["text_lower"].apply(lambda text: remove_punctuation(text))
df.head()

,text,text_lower,text_wo_punct
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...


# Removal of stopwords

In [10]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

STOPWORDS = set(stopwords.words('english'))

In [11]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [12]:
df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

,text,text_lower,text_wo_punct,text_wo_stop
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...,applesupport causing reply disregarded tapped ...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...,105835 business means lot us please dm name zi...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...,105836 livechat online moment httpstcosy94vtu8...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...


# Removal of Frequent words

In [13]:
from collections import Counter
cnt = Counter()

In [14]:
for text in df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

In [15]:
cnt.most_common(10)

[('us', 25),
 ('dm', 19),
 ('help', 18),
 ('thanks', 13),
 ('httpstcogdrqu22ypt', 12),
 ('applesupport', 11),
 ('please', 11),
 ('phone', 9),
 ('hi', 9),
 ('ive', 8)]

In [16]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])

In [17]:
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

In [18]:
df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

,text,text_lower,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...,applesupport causing reply disregarded tapped ...,causing reply disregarded tapped notification ...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...,105835 business means lot us please dm name zi...,105835 business means lot name zip code additi...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...,76328 really hope change im sure wont dont,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...,105836 livechat online moment httpstcosy94vtu8...,105836 livechat online moment httpstcosy94vtu8...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...,virgintrains see attached error message tried ...


# Removal of Rare words

In [19]:
# Drop the two columns which are no more needed 
df.drop(["text_lower","text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

In [20]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])

In [21]:
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [22]:
df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare
0,@AppleSupport causing the reply to be disregar...,causing reply disregarded tapped notification ...,causing reply disregarded tapped notification ...
1,@105835 Your business means a lot to us. Pleas...,105835 business means lot name zip code additi...,105835 business means lot name zip code additi...
2,@76328 I really hope you all change but I'm su...,76328 really hope change im sure wont dont,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu8...,105836 livechat online moment httpstcosy94vtu8...
4,@VirginTrains see attached error message. I've...,virgintrains see attached error message tried ...,virgintrains see attached error message tried ...


# Stemming

In [23]:
from nltk.stem.porter import PorterStemmer

In [24]:
# Drop the two columns 
#df.drop(["text_wo_stopfreq"], axis=1, inplace=True) 

In [25]:
stemmer = PorterStemmer()

In [26]:
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [27]:
df["text_stemmed"] = df["text_wo_stopfreqrare"].apply(lambda text: stem_words(text))
df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare,text_stemmed
0,@AppleSupport causing the reply to be disregar...,causing reply disregarded tapped notification ...,causing reply disregarded tapped notification ...,caus repli disregard tap notif keyboard opened😡😡😡
1,@105835 Your business means a lot to us. Pleas...,105835 business means lot name zip code additi...,105835 business means lot name zip code additi...,105835 busi mean lot name zip code addit detai...
2,@76328 I really hope you all change but I'm su...,76328 really hope change im sure wont dont,76328 really hope change im sure wont dont,76328 realli hope chang im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu8...,105836 livechat online moment httpstcosy94vtu8...,105836 livechat onlin moment httpstcosy94vtu8k...
4,@VirginTrains see attached error message. I've...,virgintrains see attached error message tried ...,virgintrains see attached error message tried ...,virgintrain see attach error messag tri leav v...


In [28]:
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

# Lemmatization

In [29]:
from nltk.stem import WordNetLemmatizer

In [30]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [31]:
df["text_lemmatized"] = df["text_stemmed"].apply(lambda text: lemmatize_words(text))
df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare,text_stemmed,text_lemmatized
0,@AppleSupport causing the reply to be disregar...,causing reply disregarded tapped notification ...,causing reply disregarded tapped notification ...,caus repli disregard tap notif keyboard opened😡😡😡,caus repli disregard tap notif keyboard opened😡😡😡
1,@105835 Your business means a lot to us. Pleas...,105835 business means lot name zip code additi...,105835 business means lot name zip code additi...,105835 busi mean lot name zip code addit detai...,105835 busi mean lot name zip code addit detai...
2,@76328 I really hope you all change but I'm su...,76328 really hope change im sure wont dont,76328 really hope change im sure wont dont,76328 realli hope chang im sure wont dont,76328 realli hope chang im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu8...,105836 livechat online moment httpstcosy94vtu8...,105836 livechat onlin moment httpstcosy94vtu8k...,105836 livechat onlin moment httpstcosy94vtu8k...
4,@VirginTrains see attached error message. I've...,virgintrains see attached error message tried ...,virgintrains see attached error message tried ...,virgintrain see attach error messag tri leav v...,virgintrain see attach error messag tri leav v...


In [32]:
lemmatizer.lemmatize("running")

'running'

In [33]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [34]:
remove_emoji("game is on 🔥🔥")

'game is on '

In [35]:
remove_emoji("Hilarious😂")

'Hilarious'

# Removal of URLs

In [36]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [37]:
text = "Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/"
remove_urls(text)

'Driverless AI NLP blog post on '

In [38]:
text = "Please refer to link http://lnkd.in/ecnt5yC for the paper"
remove_urls(text)

'Please refer to link  for the paper'

In [39]:
text = "Want to know more. Checkout www.h2o.ai for additional information"
remove_urls(text)

'Want to know more. Checkout  for additional information'

# Removal of HTML Tags

In [40]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [41]:
text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>"""

In [42]:
print(remove_html(text))


 H2O
 AutoML
 Driverless AI



In [43]:
#shape of the dataframe
print('The shape of the dataframe is :',df.shape)

#first few records
df.head()

#Checking for null values in `description`
df['text'].isnull().sum()


# Converting the Text to Lower Case
df['text']= df['text'].str.lower()
df['text'].head()

# Splitting and Removing Punctuation from the Text
splitted = df['text'].str.split(' ')
splitted.head()


cleaned_data = []

for text in splitted:
    text = [x.strip(string.punctuation) for x in text]
    cleaned_data.append(text)

cleaned_data[0]

text_spam = [" ".join(text) for text in cleaned_data]
final_text = " ".join(text_spam)
final_text[:500]


The shape of the dataframe is : (93, 5)


"applesupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡 105835 your business means a lot to us please dm your name zip code and additional details about your concern rr https://t.co/znuu1vjn9r 76328 i really hope you all change but i'm sure you won't because you don't have to 105836 livechat is online at the moment  https://t.co/sy94vtu8kq or contact 03331 031 031 option 1 4 3 leave a message to request a call back virgintrains see attached er"